In [ ]:
#   for Azure
#!pip install openai==0.28

#   for OpenAI
!pip install openai==1.0.0


In [1]:
import os
import openai

#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv())

#Only for Azure
#os.environ["OPENAI_API_TYPE"] = os.getenv("OPENAI_API_TYPE")
#Only for Azure
#os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#Only for Azure
#os.environ["OPENAI_API_BASE"] = os.getenv('OPENAI_API_BASE')
#Only for Azure
#os.environ["OPENAI_API_VERSION"] = os.getenv('OPENAI_API_VERSION')
openai.api_key=os.getenv('OPENAI_API_KEY')

In [ ]:
#AZURE VERSION
    
def get_completion(prompt, model="gpt-4-32k"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine="gpt-4-32k",
        messages=messages,
        temperature=0
    )

    return response.choices[0].message.content

In [2]:
#OPENAI version NO AZURE
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## Principios del Prompt Engineering
- **Principio 1: Escribir instrucciones claras y específicas**
- **Principio 2: Dar al modelo tiempo para "pensar"**

### Técnicas principio 1

#### Técnica 1: Usar delimitadores claros para indicar distintas partes del input.
- Los Delimitadores pueden ser algo tipo: ```, """, < >, `<tag> </tag>`, `:`

In [3]:
text = f"""
Debe expresar lo que quiere que haga un modelo proporcionando instrucciones que sean lo más claras y específicas posible.
Esto guiará el modelo hacia el resultado deseado y reducirá las posibilidades de recibir respuestas irrelevantes o incorrectas. No confunda escribir un mensaje claro con escribir un mensaje breve.
En muchos casos, las indicaciones más largas brindan más claridad y contexto para el modelo, lo que puede conducir a resultados más detallados y relevantes.
"""
prompt = f"""
Resume el texto delimitado por triples comillas en una sola frase.
'''{text}'''
"""
response = get_completion(prompt)
print(response)

Es importante proporcionar instrucciones claras y específicas a un modelo para obtener resultados deseados y evitar respuestas irrelevantes o incorrectas.


#### Técnica 2: Preguntar por un output estructurado
- JSON, HTML

In [ ]:
prompt = f"""
Genere una lista de tres títulos de libros inventados junto con sus autores y géneros.
Proporciónelos en formato JSON con las siguientes claves:
book_id, título, autor, género.
"""
response = get_completion(prompt)
print(response)

#### Técnica 3: Preguntar al modelo si las condiciones se cumplen

In [4]:
text_1 = f"""
¡Preparar una taza de té es fácil! Primero, necesitas poner un poco de agua a hervir. Mientras tanto, coge una taza y ponle una bolsita de té.
Una vez que el agua esté lo suficientemente caliente, simplemente viértela sobre la bolsita de té.
Déjalo reposar un rato para que el té se repose. Después de unos minutos, saca la bolsita de té.
Si quieres puedes añadir un poco de azúcar o leche al gusto.
¡Y eso es todo! Tienes una deliciosa taza de té para disfrutar.
"""
prompt = f"""
Se proporcionará un texto delimitado por comillas triples.
Si contiene una secuencia de instrucciones, vuelve a escribir esas instrucciones en el siguiente formato:

Paso 1 - ...
Paso 2 - …
…
Paso N -…

Si el texto no contiene una secuencia de instrucciones, simplemente escribe "Lo siento, no se han proporcionan pasos".

'''{text_1}'''
"""
response = get_completion(prompt)
print(response)

Paso 1 - Poner un poco de agua a hervir.
Paso 2 - Coger una taza y ponerle una bolsita de té.
Paso 3 - Verter el agua caliente sobre la bolsita de té.
Paso 4 - Dejar reposar el té durante unos minutos.
Paso 5 - Sacar la bolsita de té.
Paso 6 - Añadir azúcar o leche al gusto.
Paso 7 - Disfrutar de una deliciosa taza de té.


In [ ]:
text_2 = f"""
Hoy el sol brilla intensamente y los pájaros cantan.
Es un hermoso día para dar un paseo por el parque.
Las flores están floreciendo y los árboles se mecen suavemente con la brisa.
La gente está fuera de casa, disfrutando del hermoso clima.
Algunos hacen picnic, mientras que otros juegan o simplemente se relajan en el césped.
Es un día perfecto para pasar tiempo al aire libre y apreciar la belleza de la naturaleza.
"""
prompt = f"""
Se proporcionará un texto delimitado por comillas triples.
Si contiene una secuencia de instrucciones, vuelve a escribir esas instrucciones en el siguiente formato:

Paso 1 - ...
Paso 2 - …
…
Paso N -…

Si el texto no contiene una secuencia de instrucciones, simplemente escribe "Lo siento, no se han proporcionan pasos".

'''{text_2}'''
"""
response = get_completion(prompt)
print(response)

#### Técnica 4: "Few-shot" prompting

In [ ]:
prompt = f"""
Su tarea es responder con un estilo coherente.

<niño>: Enséñame sobre la paciencia.

<abuelo>: El río que labra el valle más profundo brota de un modesto manantial;
la sinfonía más grandiosa se origina a partir de una sola nota;
El tapiz más complejo comienza con un hilo solitario.

<niño>: Enséñame sobre la resiliencia.
"""
response = get_completion(prompt)
print(response)

### Técnicas principio 2

#### Técnica 1: Especificar los pasos necesarios para finalizar una tarea.

In [ ]:
text = f"""
En un pueblo encantador, los hermanos Jack y Jill emprenden una búsqueda para buscar agua de un pozo en la cima de una colina.
Mientras subían, cantando alegremente, llegó la desgracia: Jack tropezó con una piedra y cayó colina abajo, y Jill se tiró a intentar rescatarlo.
Aunque un poco golpeados, la pareja regresó a casa. A pesar del percance, su espíritu aventurero permaneció intacto y continuaron explorando con ganas.
"""
# example 1
prompt_1 = f"""
Realiza las siguientes acciones:
1 - Resume el siguiente texto delimitado por triples comillas invertidas con 1 frase.
2 - Traducir el resumen al inglés.
3 - Enumera cada nombre en el resumen en inglés.
4 - Genera un objeto json que contenga las siguiente claves: resumen_inglés, num_nombres.

Separa sus respuestas con saltos de línea.

Texto:
```{text}```
"""
response = get_completion(prompt_1)
print(response)

#### Técnica 2: Indicar al modelo que encuentre su propia solución antes de apresurarse a llegar a una conclusión.

In [ ]:
prompt = f"""
Determinar si la solución del estudiante es correcta o no.

Pregunta:
Estoy construyendo una instalación de energía solar y necesito ayuda para resolver las finanzas.
- El terreno cuesta $100 / metro cuadrado
- Puedo comprar paneles solares por $250 / metro cuadrado
- Negocié un contrato de mantenimiento que costará 100.000 dólares fijos al año y 10 dólares adicionales por metro cuadrado
¿Cuál es el coste total del primer año de operaciones en función del número de metros cuadrados?.

Solución del estudiante:
Sea x el tamaño de la instalación en pies cuadrados.
Costes:
1. Coste del terreno: 100x
2. Coste del panel solar: 250x
3. Coste de mantenimiento: 100.000 + 100x
Coste total: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

Podemos mejorar esta solución dándole instrucciones al modelo para que trabaje primero en su propia solución y luego la compare con la del alumno.

In [ ]:
prompt = f"""
La tarea es determinar si la solución del estudiante es correcta o no.
Para resolver el problema haz lo siguiente:
- Primero, elabora tu propia solución al problema, incluido el total final.
- Luego compara tu solución con la solución del estudiante y evalúa si la solución del estudiante es correcta o no.
No decidas si la solución del estudiante es correcta hasta que tu mismo hayas resuelto el problema.

Utiliza el siguiente formato:
Pregunta:
```
pregunta aquí
```
La solución del estudiante:
```
la solución del estudiante aquí
```
Solución real:
```
la solución correcta aquí
```

¿La solución del estudiante es la misma que la solución real?

```
sí o no
```
Calificación del estudiante:
```
correcto o incorrecto
```
La solución real
```
Escribe aquí la solución real
```

Pregunta:
```
Estoy construyendo una instalación de energía solar y necesito ayuda para resolver las finanzas.
- El terreno cuesta $100 / metro cuadrado
- Puedo comprar paneles solares por $250 / metro cuadrado
- Negocié un contrato de mantenimiento que costará 100.000 dólares fijos al año y 10 dólares adicionales por metro cuadrado
¿Cuál es el coste total del primer año de operaciones en función del número de metros cuadrados?.
```
La solución del estudiante:
```
Sea x el tamaño de la instalación en pies cuadrados.
Costes:
1. Coste del terreno: 100x
2. Coste del panel solar: 250x
3. Coste de mantenimiento: 100.000 + 100x
Coste total: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Solución real:
```
pasos para encontrar la solución y tu solución aquí
```
"""
response = get_completion(prompt)
print(response)

## Prompt development iterativo
- **En este ejercicio, vamos a analizar y refinar nuestros prompts de forma iterativa, con el objetivo de generarar un texto enfocado a marketing sobre una ficha técnica de un producto**

### Generar una descripción de marketing de la ficha técnica de un producto

In [ ]:
fact_sheet_chair = """
DESCRIPCIÓN GENERAL
- Parte de una hermosa familia de muebles de oficina inspirados en mediados de siglo incluidos archivadores, escritorios, estanterías, mesas de reuniones y más.
- Varias opciones de color de carcasa y acabados de base.
- Disponible con tapizado trasero y delantero de plástico (SWC-100) o tapizado completo (SWC-110) en 10 opciones de tela y 6 de cuero.
- Las opciones de acabado de la base son: acero inoxidable, negro mate, blanco brillo o cromado.
- Silla disponible con o sin reposabrazos.
- Adecuado para entornos domésticos o comerciales.

CONSTRUCCIÓN
- Base de aluminio plastificado de 5 ruedas.
- Ajuste neumático de la silla para facilitar la acción de subir/bajar.

DIMENSIONES
- ANCHO 53 CM | 20,87”
- PROFUNDIDAD 51 CM | 20.08”
- ALTURA 80 CM | 31.50”
- ALTURA DEL ASIENTO 44 CM | 17,32”
- PROFUNDIDAD DEL ASIENTO 41 CM | 16,14”

OPCIONES
- Opciones de ruedas para piso blando o duro.
- Dos opciones de densidades de espuma de asiento:
  medio (1,8 lb/ft3) o alto (2,8 lb/ft3)
- Reposabrazos de PU sin brazos o de 8 posiciones

MATERIALES
PLANEADOR CON BASE DE CONCHA
- Aluminio fundido con revestimiento de nailon modificado PA6/PA66.
- Grosor de la carcasa: 10 mm.
ASIENTO
- Espuma HD36

PAÍS DE ORIGEN
- Italia
"""

prompt = f"""
La tarea es ayudar a un equipo de marketing a crear una
descripción en un sitio web minorista de un producto basado
en una ficha técnica.

Escribe una descripción del producto basada en la información.
prevista en las especificaciones técnicas delimitadas por
triples comillas invertidas.

Especificaciones técnicas: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

### Primer reto: El texto es demasiado largo. Debemos limitar el número de palabras o frases

In [ ]:
prompt = f"""
La tarea es ayudar a un equipo de marketing a crear una
descripción en un sitio web minorista de un producto basado
en una ficha técnica.

Escribe una descripción del producto basada en la información.
prevista en las especificaciones técnicas delimitadas por
triples comillas invertidas.

Utiliza como máximo 50 palabras.

Especificaciones técnicas: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

### Segundo reto: La descripción se enfoca en algunos detalles no relevantes para la audiencia, que son minoristas de muebles.

In [ ]:
prompt = f"""
La tarea consiste ayudar a un equipo de marketing a crear una
descripción en un sitio web minorista de un producto basado
en una ficha técnica.

Escribe una descripción del producto basada en la información.
prevista en las especificaciones técnicas delimitadas por
triples comillas invertidas.

La descripción está destinada a minoristas de muebles,
por lo que debería ser de naturaleza técnica y centrarse en la
materiales con los que está construido el producto.

Utilice como máximo 50 palabras.

Especificaciones técnicas: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
La tarea consiste en ayudar a un equipo de marketing a crear una
descripción en un sitio web minorista de un producto basado
en una ficha técnica.

Escribe una descripción del producto basada en la información.
prevista en las especificaciones técnicas delimitadas por
triples comillas invertidas.

La descripción está destinada a minoristas de muebles,
por lo que debería ser de naturaleza técnica y centrarse en la
materiales con los que está construido el producto.

Al final de la descripción, incluye todos los ID del producto compuestos por 7 caracteres que aparecen en la especificación técnica.

Utilice como máximo 50 palabras.

Especificaciones técnicas: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

### Tercer reto: La descripción necesita una tabla de dimensiones.

In [ ]:
prompt = f"""
La tarea consiste en ayudar a un equipo de marketing a crear una
descripción de para un sitio web minorista de un producto basado
en una ficha técnica.

Escribe una descripción del producto basada en la información.
previsto en las especificaciones técnicas delimitadas por
triples comillas invertidas.

La descripción está destinada a minoristas de muebles,
por lo que debería ser de naturaleza técnica y centrarse en la
materiales con los que está construido el producto.

Al final de la descripción, incluye todos los ID del producto compuestos por 7 caracteres que aparecen en la especificación técnica.


Después de la descripción, incluye una tabla que proporcione todas las dimensiones del producto. La tabla debe tener dos columnas.
En la primera columna incluye el nombre de la dimensión.
En la segunda columna incluye las medidas solo en pulgadas.

Asigna a la tabla el título "Dimensiones del producto".

Formatea todo como HTML que pueda usarse en un sitio web.
Coloca la descripción en un elemento <div>.

Especificaciones técnicas: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

## Prompt development resúmenes
- **En este ejercicio, vamos a resumir textos enfocados en diferentes temáticas**

### Texto para resumir con un límite de palabras o frases.

In [ ]:
prod_review = """
Conseguí este oso panda de peluche para el cumpleaños de mi hija que lo ama y lo lleva a todas partes.
Es suave y súper lindo y su cara tiene una mirada amigable.
Es un poco pequeño para lo que pagué. Creo que podrían haber otros osos que sean más grandes para ese mismo precio.
Llegó un día antes de lo esperado, así que pude jugar con él antes de dárselo a ella.
"""
prompt = f"""
La tarea consiste en generar un breve resumen de un producto reseña de un sitio de comercio electrónico.

Resume la reseña a continuación, delimitada por triple comillas invertidas, en un máximo de 30 palabras.

Resumen: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

### Hacer un resumen del texto enfocado en el transporte y la entrega

In [ ]:
prompt = f"""
La tarea consiste en generar un breve resumen de un producto, revisar desde un sitio de comercio electrónico para dar su opinión al departamento de envíos.

Resume la reseña a continuación, delimitada por triples comillas invertidas, en un máximo de 30 palabras y centrándose en cualquier aspecto \
que mencione el envío y entrega del producto.

Resumen: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

### Hacer un resumen del texto enfocado en el precio y valor aportado

In [ ]:
prompt = f"""
La tarea consiste en generar un breve resumen de un producto, revisarlo desde un sitio de comercio electrónico para dar su opinión al departamento de ventas,
responsable de determinar el precio del producto.

Resume la reseña a continuación, delimitada por triples comillas invertidas,
en un máximo de 30 palabras y centrándose en cualquier aspecto que sea relevante para el precio y el valor percibido.

Resumen: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

### Resumir múltiples reseñas al mismo tiempo

In [ ]:
review_1 = prod_review 

# review for a standing lamp
review_2 = """
Necesitaba una bonita lámpara para mi dormitorio y que tuviese un precio no demasiado alto.
Lo conseguí rápido, llegó en 2 días.
La cuerda se le rompió durante el transporte y la empresa me ha enviado uno nuevo.
Llegó a los pocos días también. Fue fácil de montar.
Entonces tuve un problema y es que faltaba una parte, así que me comuniqué con su soporte y ellos me consiguieron 
la pieza que faltaba de forma rápida.
Es una gran empresa que se preocupa por sus clientes y productos. 
"""

# review for an electric toothbrush
review_3 = """
Mi dentista me recomendó un cepillo de dientes eléctrico.
Por eso compré esto. La duración de la batería parece ser bastante impresionante hasta ahora.
Después de la carga inicial y dejar el cargador enchufado durante la primera semana para acondicionar la batería,
he desconectado el cargador y lo he estado usando para cepillarme tres veces al día durante 
3 semanas todo con la misma carga. 
Sin embargo, el cabezal del cepillo de dientes es demasiado pequeño. He visto cepillos de dientes para bebés más grandes que este.
Ojalá la cabeza fuera más grande con diferentes hilos de longitud para llegar mejor entre los dientes porque este no es capaz.
En general, si puedes conseguir este cepillo alrededor de los 50 dólares, es una buena oferta.
El fabricante de los cabezales de repuesto es bastante caro, pero puedes obtener genéricos que tengan un precio más razonable. 
Este cepillo de dientes me hace sentir como si hubiera ido al dentista cada día. ¡Mis dientes se sienten relucientes y limpios!
"""

reviews = [review_1, review_2, review_3]

for i in range(len(reviews)):
    prompt = f"""
    La tarea consiste en generar un breve resumen de un producto/reseña de un sitio de comercio electrónico.

    Resume la reseña a continuación, delimitada por triples comillas invertidas en un máximo de 20 palabras.

    Resumen: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

## Prompt development inferencia
- **En este ejercicio, vamos a inferir sentimiento y temáticas de las reseñas y noticias sobre artículos**

### Clasificación de sentimiento (positivo o negativo)

In [ ]:
lamp_review = """
Necesitaba una bonita lámpara para mi dormitorio y que tuviese un precio no demasiado alto.
Lo conseguí rápido, llegó en 2 días.
La cuerda se le rompió durante el transporte y la empresa me ha enviado uno nuevo.
Llegó a los pocos días también. Fue fácil de montar.
Entonces tuve un problema y es que faltaba una parte, así que me comuniqué con su soporte y ellos me consiguieron 
la pieza que faltaba de forma rápida.
Es una gran empresa que se preocupa por sus clientes y productos. 
"""

prompt = f"""
¿Cuál es el sentimiento de la siguiente revisión del producto?

Texto de la reseña: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
¿Cuál es el sentimiento de la siguiente revisión del producto?

Da una respuesta empleando una única palabra, ya sea "positiva" o "negativa".

Texto de la reseña: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### Identificación de tipos de emociones

In [ ]:
prompt = f"""
Identifica una lista de emociones expresadas por el escritor del la siguiente reseña.
No incluyas más de cinco elementos de la lista.
Formatea tu respuesta como una lista de palabras en minúscula separadas por comas.

Texto de la reseña: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### Identificación de enojo

In [ ]:
prompt = f"""
¿El autor de la siguiente reseña está expresando enojo?
La reseña está delimitada con triples comillas.
Dá tu respuesta como sí o no.

Texto de la reseña: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### Extracción de los productos y el nombre de la empresa de las reseñas de los clientes

In [ ]:
prompt = f"""
Identifica los siguientes elementos del texto de revisión:
- Artículo comprado por el cliente
- Empresa que fabricó el artículo.

La reseña está delimitada con triples comillas.
Formatea tu respuesta como un objeto JSON con "Artículo" y "Marca" como claves.
Si la información no está presente, utiliza "desconocido" como valor.
Haz tu respuesta lo más breve posible.
  
Texto de la reseña: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### Realizar múltiples tareas al mismo tiempo

In [ ]:
prompt = f"""
Identifica los siguientes elementos del texto de revisión:
- Sentimiento (positivo o negativo)
- ¿El crítico expresa enojo? (verdadero o falso)
- Artículo comprado por el crítico
- Empresa que fabricó el artículo.

La reseña está delimitada con triples comillas.
Formatea tu respuesta como un objeto JSON con "Sentimiento", "Ira", "Artículo" y "Marca" como claves.
Si la información no está presente, utiliza "desconocido" como valor.
Haz tu respuesta lo más breve posible.
Formatea el valor de Ira como booleano.

Texto de la reseña: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### Infiriendo temáticas

In [ ]:
story = """
En una encuesta reciente realizada por el gobierno,
se pidió a los empleados del sector público que calificaran su nivel de satisfacción con el departamento en el que trabajan.
Los resultados revelaron que la NASA era el departamento más popular con un índice de satisfacción del 95%.

Un empleado de la NASA, John Smith, comentó sobre los hallazgos afirmando: "No me sorprende que la NASA haya salido victoriosa.
Es un gran lugar para trabajar con gente increíble y
oportunidades increíbles. Estoy orgulloso de ser parte de
una organización tan innovadora."

Los resultados también fueron bien recibidos por el equipo directivo de la NASA,
con el director Tom Johnson afirmando: "Estamos encantados de poder escuchar
que nuestros empleados están satisfechos con su trabajo en la NASA.
Contamos con un equipo talentoso y dedicado que trabaja incansablemente.
para lograr nuestros objetivos, y es fantástico ver que su trabajo duro da sus frutos."

La encuesta también reveló que el departamento de administración de la seguridad social tuvo la satisfacción más baja
y comunicaron que tan solo el 45% de los empleados indicando que estaban
satisfechos con su trabajo. 
El gobierno se ha comprometido a abordar las inquietudes planteadas por los empleados en la encuesta y
a trabajar para mejorar la satisfacción laboral en todos los departamentos.
"""

prompt = f"""
Determina cinco temas que se están discutiendo en el siguiente texto, que está delimitado por triples comillas.

Haz que cada elemento tenga una o dos palabras.

Formatea tu respuesta como una lista de elementos separados por comas.

Textos de ejemplo: '''{story}'''
"""
response = get_completion(prompt)
print(response)

## Prompt development transformación
- **En este ejercicio, vamos a explorar cómo utilizar LLMs para tareas de transformación de texto, como traducción de idiomas, revisión ortográfica y gramatical, ajuste de tono y conversión de formato.**

### Traducción

In [ ]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Dime en qué idioma está escrita la frase entre comillas triples.
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basket ball```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

### Transformación de tono. La redacción de texos puede variar según la audiencia prevista. Los modelos GPT pueden producir diferentes tonos.

In [ ]:
prompt = f"""
Traduce lo siguiente de jerga callejera a un tono comercial:
'Tronco, soy Jorge, échale un ojo a las especificaciones de esta lámpara de pie y dime algo.'
"""
response = get_completion(prompt)
print(response)

### Transformación de formato

In [ ]:
data_json = { "Empleados del restaurante" :[ 
    {"name":"Diego", "email":"diego@gmail.com"},
    {"name":"Julia", "email":"julia@gmail.com"},
    {"name":"Susana", "email":"susana@gmail.com"}
]}

prompt = f"""
Traduce el siguiente diccionario de Python en una tabla HTML con columnas y título: {data_json}
"""
response = get_completion(prompt)
print(response)

### Verificación gramatical

In [ ]:
text = [ 
  "Ba a ser un día largo",  # V no B
  "Natalia le pegó por no cumplir sus orden", # error en plural/singular
  "Me hecha de menos" # sin H
]
for t in text:
    prompt = f"""Revisa y corrige el siguiente texto.
     Reescribe la versión corregida. Si no encuentras los errores, simplemente di "No se encontraron errores". 
     El texto se encuentra entre comillas triples:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

## Prompt development ++ Bonus
- **En este ejercicio, vamos a generar emails de atención al cliente personalizados en base a la reseña de cada cliente**

In [ ]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negativo"

# review for a blender
review = f"""
Entonces, todavía tenían el sistema de 17 piezas a la venta por alrededor de $49 en el mes de noviembre,
aproximadamente a mitad de precio, pero por alguna razón (llámelo aumento abusivo de precios),
alrededor de la segunda semana de diciembre todos los precios subieron hasta aproximadamente entre $ 70 y $ 89 por el mismo sistema.
El sistema de 11 piezas subió alrededor de $10 por lo que el precio también difiere del precio de venta anterior de 29 dólares.
Se ve bien, pero si miras la base, la parte donde la hoja encaja en su lugar no se ve tan bien
como en ediciones anteriores de hace unos años, pero yo planeo usar este aparato para tareas sencillas
(por ejemplo, aplastar alimentos muy duros como frijoles, hielo, arroz, etc. en la licuadora primero
y luego pulverizarlos en el tamaño de la porción).
Consejo especial al hacer batidos, cortar finamente y congelar las frutas y
verduras (si usa espinacas, guisadas ligeramente, ablanda las
espinacas y luego congélalas hasta que estén listas para usar, y si las preparas para
sorbete, usa un procesador de alimentos de tamaño pequeño a mediano)
Después de aproximadamente un año, el motor hacía un ruido extraño, llamé a atención al cliente pero la garantía expiró.
Por este motivo tuve que comprar otro. 
Para vuestra información: la calidad general es buena en este tipo de productos, por eso
en cierto modo cuentan con el reconocimiento de la marca y fidelización del consumidor para mantener las ventas.
Me llegó en aproximadamente dos días.
"""

prompt = f"""
Eres un asistente de IA de servicio al cliente.
Tu tarea es enviar una respuesta por correo electrónico a un cliente valioso.
Dado el correo electrónico del cliente delimitado por ```, tienes que generar una respuesta para agradecer al cliente su reseña.
Si el sentimiento es positivo o neutral, agradéceles su reseña.
Si el sentimiento es negativo, discúlpate y sugiere que pueden comunicarse con el servicio de atención al cliente.
Asegúrate de utilizar detalles específicos de la revisión.
Escribe en un tono conciso y profesional.
Firma el correo electrónico como "agente de cliente de AI".
Reseña del cliente: ```{review}```
Sentimiento de la reseña: {sentiment}
"""
response = get_completion(prompt)
print(response)